# This script generates data for Cassandra.

Usually it will take around 20-25 mins for loading the data again depends on system and number of workers for Cassandra.

Since we just have 1 worker, it can take up sometime for bulk loading the data.

In [ ]:
from cassandra.cluster import Cluster
import pandas as pd

In [ ]:
cluster = Cluster(['127.0.0.1'],port=9042)

In [ ]:
session = cluster.connect()

In [ ]:
session.execute("DROP KEYSPACE IF EXISTS chats")

In [ ]:
session.execute("CREATE KEYSPACE IF NOT EXISTS chats WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 }")

In [ ]:
session.execute('''use chats''')

In [ ]:
session.execute("DROP TABLE IF EXISTS messages")

In [ ]:
session.execute('''CREATE TABLE messages (
  channel_id text,
  time_sent timestamp,
  message_id timeuuid,
  author_id text,
  messsage text,
  PRIMARY KEY (channel_id, message_id)
) WITH CLUSTERING ORDER BY (message_id DESC);''')

In [ ]:
msgs = pd.read_csv("cassandra_data.csv")

In [ ]:
for index, row in msgs.iterrows():
    if index % 10000 == 0:
        print(index)
    query = '''INSERT INTO messages (channel_id,time_sent,message_id,author_id,messsage) VALUES ('%s', toTimestamp(now()), now(),'%s','%s');'''\
                    % (row['channel_id'],row['author_id'],row['messsage'])
    
    session.execute(query)